---
jupyter: python3
---

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sb
import statsmodels.api as sm
import sklearn.tree as tr
import sklearn.ensemble as ens
import sklearn.linear_model as slm
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

In [ ]:
nfl = pd.read_csv("./NFL_play_by_play_2022.csv.gz")
nfl.shape

These data record play-by-play information for all games in the 2022 National Football League (NFL) season. These data were downloaded using the `nflverse` package for the R programming language (another statistics and data science environment), lightly edited, and saved in a tabular format for us to use in Python.

There are many measurements for each play, some of which are computed values from `nflverse`. Here's a brief list using the data dictionary.

In [ ]:
nfl_data_dictionary = pd.read_csv("./NFL_play_by_play_data_dictonary.csv", index_col = "Field")
nfl_data_dictionary.loc[["play_id", "game_id", "home_team", "away_team", "posteam",
                         "defteam", "yardline_100", "down", "ydstogo",
                        "touchdown", "play_type"]]

## Question 1

### Part (a)

For this section, we will aggregate the individual plays into games.

Use `groupby` on `"game_id"` to aggregate(`.agg`) the games. Include the following columns:

```
{"home_score": "first",
 "away_score": "first",
 "week": "first",
 "home_team": "first",
 "away_team": "first",
 "roof": "first",
 "wind": "median",
 "temp": "median",
 "play_id": "size"}
```

Call the result `games`. Demonstrate using a plot that shows the number of games played each week. The season is composed of a regular season in which all teams play and post season playoffs in which only some teams play. Using the plot, how many weeks are in a regular season?



### Part (b)

Some people think teams benefit from playing at home. Compute the difference between the home team score and the away team score and store it as a new column (call it `"home_away_score"`).

Plot this new variable. Do you see evidence of this claim?
### Part (c)

Suppose these games represent a sample from all possible games that could have been played in 2022. Let $X$ be the home and away teams' score difference. Test the hypothesis:

$H_0: E(X) = 0$ against $H_1: E(X) \ne 0$

at the 5% level or create a 95% confidence interval for $E(X)$. What do you conclude about this hypothesis. Interpret it as evidence for or against the claim of home field advantage.


### Part (d)

One theory of home game advantage states that teams that play outdoors in cold weather are acclimated to cold weather, while teams that do not play outdoors will not perform as well in outdoor games.

We will ask a slightly simpler question and ask if the average home and away difference in outdoor games is larger than in indoor games.

To do this, we need to identify if a game is played outdoors. Investigate the `"roof"` column and create a new column (call it `"is_outdoors"`) that has the value True if the games is played outdoors and False otherwise.


Use a box plot to explore whether games played outdoors have different home and away score differences than non-outdoor games.


### Part (e)

Perform a difference of means hypothesis test to the the hypothesis that the average score difference is the same for both outdoors and non-outdoors games against the alternative that it is different. At the 5% level (or using a 95% confidence interval) what do you conclude?



### Part (f)

Another way to perform this test is to use linear regression. If we write:

$$E(Y \mid X = x) = a + b x$$

Then the difference of means for $$E(Y \mid X = 1) - E(Y \mid X = 0) = (a + b ) - (a + b \cdot 0) = b$$

The hypothesis test will use a slightly different standard error calculation, but it will be still be a valid way to test this hypothesis or get confidence intervals.

Use the `sm.OLS` to perform a linear regression of `"home_away_score`" on `"is_outdoors"`. You will need to convert the `"is_outdoors"` variable to a numeric 1/0 version first. This can be done by using `.astype('int')` to create a new column of 0 and 1 values.

Display the confidence intervals for each coefficient. For the `is_outdoors` coefficient, what do you see?

In [ ]:
## quick example for conversion
tf = pd.Series([True, False, False, True])
tf.astype("int")

### Part (g)

If our theory that outdoor games helps the home team because of the weather, perhaps we can use measured temperature and wind to see if decreasing temperature and increasing wind increases the the home team's score over the away team.

You will notice that there is some amount of missingness for the `"temp"` and `"wind"` columns. Create a new column that track if either are missing for each game.

Compute the conditional probability of missing either of these measurements for the different `"roof"` categories. What do you notice?



### Part (h)

Perform a multiple linear regression using `"is_outdoors"` (converted to 0 and 1), `"wind"`, and `"temp`". Print out the parameters and 95% confidence intervals.

For each factor, holding the others constant, would we reject the hypothesis that the conditional mean of the score difference is independent of the factor?


## Question 2

In this question, we will look at the relationship between different types of plays (passing the ball, running the ball) and the "down" (the 4 attempts the offensive team has to gain 10 yards before turning over the ball to the other side).

Most plays are either passing or running the ball. When teams are on their 4th down and do not think they can make the full 10 years, they will often punt it. Because this almost only happens on 4th downs and several of the other play types are so specialized, we will focus on just runs and passes.

In [ ]:
plays = nfl.loc[nfl["play_type"].isin(["run", "pass"])].dropna(subset = ["play_type", "down"])
plays["play_type"].value_counts()

We will relate this to the "down" column to see where runs and passes are more common.

In [ ]:
plays["down"].describe()

### Part (a)

 Calculate the empirical joint probabilities of `play_type` and `down` in the `plays` table. Also compute the counts (we will use both later).

Look at the results. Do you notice anywhere that the patterns in one column are not the same as the patterns in the other column?

Hint: You may find it helpful to use `.value_counts()` with both `normalize=True` and `normalize=False`. Additionally, if there are combinations with no occurrences, `.value_counts()` will report a `NaN`. We can use `.fillna(0)` to replace these `NaN`s with `0`s.


### Part (b)

We are ultimately concerned with whether the two variables, `play_type` and `down` are independent. Assuming that they are indeed independent (i.e. under the null hypothesis), calculate their joint probability using the product of the marginals.


Then, multiply these joint probabilities by the number of observations to obtain the expected counts.


### Part (c)

First, calculate and store the `pearson_residuals` by subtracting the expected counts calculated in part (b) from the observed counts calculated in part (a), and dividing by the square root of the expected counts.

Now, perform a Z-test to determine if our two variables `play_type` and `down` are independent, with an $\alpha = 0.003$. Report your findings.

### Part (d)

Now, calculate the sum of squares of Pearson residuals.

Then calculate the degrees of freedom for a Chi-squared test, and find the rejection region with $\alpha=0.003$.

Compare the sum of squares of Pearson residuals (our test statistic) to the rejection region. Are the results consistent with the Z-test we performed earlier?


## Question 3

In this problem we will use a decision tree to predict the outcome of a play. We will use the `play_type` as the outcome variable and the `down`, `ydstogo`, and `yardline_100` as the predictors.

### Part (a)

Create a new table `plays2` that only includes the columns `play_type`, `down`, `ydstogo`, and `yardline_100`. Drop any rows with missing values. Compute the marginal probabilities for the different play types.

In [ ]:
plays2 = plays[["play_type", "down", "ydstogo", "yardline_100"]].dropna()
plays2["play_type"].value_counts(normalize = True)

### Part (b)

Using `max_depth = 2, random_state = 101`, use the `DecisionTreeClassifier` from `sklearn.tree` to fit a decision tree to the data. Use the `play_type` as the outcome variable and the `down`, `ydstogo`, and `yardline_100` as the predictors.

Print out the tree using the `plot_tree` function from `sklearn.tree`. By setting the `feature_names = X.columns` and `filled = True`, you can make the tree more readable.

### Part (c)

Inspect the graph in the previous step. The boxes will have numbers like `[100, 200]`. This represents the numbers of passes and runs contained in that split, respectively. Use the graph to answer the following questions:

- What variable was used to create the first split? What is the decision made about that variable?
- For a play that is on 3rd down, with 5 yards to go, and on the 50 yard line, what is the predicted outcome?
- For what group of plays is the decision tree most confident in its prediction?

### Part (d)

Now create a tree without a maximum depth. Using the `predict` method, predict the following plays:

In [ ]:
plays_to_predict = pd.DataFrame({"down": [1, 2, 3, 4],
                                 "ydstogo": [10, 5, 2, 1],
                                 "yardline_100": [20, 50, 80, 99]})

Use the `predict_proba` method to get the probabilities of each play type for each of these plays. For which of these predictions was the tree most confident? For which was the tree the least confident?


### Part (e)

Now, use the `cross_val_score` function from `sklearn.model_selection` to perform a 5-fold cross validation on the data. Use the `DecisionTreeClassifier` with `random_state = 100` and `max_depth = 2`. Print out the mean accuracy of the model. (Larger values are better.)

Repeat this process with a `max_depth = 5`. What do you notice about the accuracy of the model as the depth increases?

### Part (f)

Using the `RandomForestClassifier` from `sklearn.ensemble`, fit a random forest model to the data with `n_estimators = 100` and `random_state = 100`. Perform a 5-fold cross validation on the data and print out the mean accuracy of the model. Which technique performs better: decision tree classifiers or random forests?
